## Populate Database

Script to take a set of data in numpy files and put it in a local database for testing / example purposes. The data are already in the production database

In [ ]:
from opdb import opdb
import numpy as np
import pandas as pd


In [ ]:
#change this to where yoru files are, and your local DB information. If working with the real 
#database you don't need this notebook. 
dataPath = "/Users/karr/Science/Data/PFS/Numpy/"
hostname = "localhost"
port = "5432"
dbname = "opdb"
username = "karr"
db=opdb.OpDB(hostname = hostname, port = port,dbname = dbname,
                        username = username)


In [ ]:

visitId=72752

# read in the numpy files

dFile = dataPath+str(int(elem))+"_data.npy"
mFile =  dataPath+str(int(elem))+"_match.npy"
tFile =  dataPath+str(int(elem))+"_target.npy"
eFile =  dataPath+str(int(elem))+"_expose.npy"
    
data = np.load(dFile,allow_pickle=True)
expose = np.load(eFile,allow_pickle=True)
target = np.load(tFile,allow_pickle=True)
match = np.load(mFile,allow_pickle=True)

# write to DB. order of tables is important

visit=np.array([elem])
columns=["pfs_visit_id"]
df=pd.DataFrame(visit,columns=columns)
db.insert("pfs_visit", df)

expose = expose[:,0:2]
columns = ["mcs_frame_id","pfs_visit_id"]
df=pd.DataFrame(expose,columns=columns)
db.insert("mcs_exposure", df)
 
columns=["pfs_visit_id","iteration","cobra_id","pfi_nominal_x_mm","pfi_nominal_y_mm","pfi_target_x_mm","pfi_target_y_mm"]
df=pd.DataFrame(target[:,[0,1,2,4,5,6,7]],columns=columns)
db.insert("cobra_target", df)

columns=["mcs_frame_id","spot_id","mcs_center_x_pix","mcs_center_y_pix","mcs_second_moment_x_pix","mcs_second_moment_y_pix","mcs_second_moment_xy_pix","bgvalue","peakvalue"]
df=pd.DataFrame(data,columns=columns)
db.insert("mcs_data", df)

columns=["pfs_visit_id","iteration","cobra_id","mcs_frame_id","spot_id","pfi_center_x_mm","pfi_center_y_mm","flags"]
df=pd.DataFrame(match,columns=columns)
db.insert("cobra_match", df)
